In [ ]:
from pathlib import Path
import pickle

import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from evo import THPProblem

In [ ]:
results_dir = Path("results/redis")

In [ ]:
with open(results_dir / "X.npy", "rb") as f:
    X = np.load(f)

with open(results_dir / "F.npy", "rb") as f:
    F = np.load(f)

with open(results_dir / "config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

with open(results_dir / "fullresults.pkl", "rb") as f:
    full_results = pickle.load(f)

In [ ]:
def get_results_df(full_results, config: dict):
    sys_params = config["sys_params"]
    objectives = config["objectives"]
    cols = ["gen"] + [param.split("/")[-1] for param in sys_params.keys()] + objectives

    df = pd.DataFrame(columns=cols)

    for gen, history in enumerate(full_results.history):
        X = history.pop.get("X")
        F = history.pop.get("F")
        gen_col = np.full((X.shape[0], 1), gen)

        cat = np.concatenate([gen_col, X, F], axis=1)

        df = pd.concat([df, pd.DataFrame(cat, columns=cols)], ignore_index=True)

    df["defrag"] = (df["defrag"] > 0.5).astype(int)
    
    return df
    

results_df = get_results_df(full_results, config)

In [ ]:
print(results_df[results_df["gen"] == 9])

In [ ]:
plt.scatter(results_df["max_ptes_none"], results_df["throughput"], c=results_df["gen"])
plt.colorbar()
plt.show()

In [ ]:
plt.scatter(results_df["throughput"], results_df["fragmentation"], c=results_df["gen"])
plt.show()

In [ ]:
full_results.history[0].pop.get("X")

In [ ]:
X.shape

In [ ]:
print(F)

In [ ]:
plt.scatter(F[:, 0], F[:, 1])
plt.xlabel("throughput")
plt.ylabel("fragmentation ratio")
plt.title("Objective Space")
plt.show()

In [ ]:
print(X)

In [ ]:
print(sys_params)

In [ ]:
plt.scatter(X[:, 1], X[:, 3])
plt.xlabel("scan_sleep_millisecs")
plt.ylabel("pages_to_scan")
plt.title("Action Space")
plt.show()